In [1]:
import torch
from torch.utils.cpp_extension import load
import numpy as np


differentiable_rapid_raditor = load(
    name="differentiable_rapid_raditor",
    sources=["utils/differentiable_rapid_raditor_kernel.cpp", "utils/differentiable_rapid_raditor_kernel_v3_fine.cu"],
    verbose=True,
)

Using C:\Users\ls\AppData\Local\torch_extensions\torch_extensions\Cache\py39_cu124 as PyTorch extensions root...
Creating extension directory C:\Users\ls\AppData\Local\torch_extensions\torch_extensions\Cache\py39_cu124\differentiable_rapid_raditor...
Detected CUDA files, patching ldflags
Emitting ninja build file C:\Users\ls\AppData\Local\torch_extensions\torch_extensions\Cache\py39_cu124\differentiable_rapid_raditor\build.ninja...
c:\Users\ls\anaconda3\envs\stimulate\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module differentiable_rapid_raditor...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module differentiable_rapid_raditor...


In [2]:
# Define the custom Autograd Function
class SimulateFunction(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, sensor_location, source_location, source_p0, source_dx, dt, num_sensors, num_sources, num_times):
        # Call the C++ forward function
        simulate_record = differentiable_rapid_raditor.simulate(
            sensor_location, source_location, source_p0, source_dx, dt, num_sensors, num_sources, num_times)
        
        # Save inputs for backward
        ctx.save_for_backward(sensor_location, source_location, source_p0, source_dx)
        ctx.dt = dt
        ctx.num_sensors = num_sensors
        ctx.num_sources = num_sources
        ctx.num_times = num_times
        
        return simulate_record  # simulate_record：torch.Size([num_sensors * num_times])
    
    @staticmethod
    def backward(ctx, dL_dsimulate_record):
        # dL_dsimulate_record：torch.Size([num_sensors * num_times])
        sensor_location, source_location, source_p0, source_dx = ctx.saved_tensors
        dt = ctx.dt
        num_sensors = ctx.num_sensors
        num_sources = ctx.num_sources
        num_times = ctx.num_times


        # Call the C++ backward function
        grad_source_location, grad_source_p0, grad_source_dx = differentiable_rapid_raditor.simulate_backward(
            sensor_location, source_location, source_p0, source_dx, dL_dsimulate_record.contiguous(), dt, num_sensors, num_sources, num_times
        )
        return None, grad_source_location, grad_source_p0, grad_source_dx, None, None, None, None

# Utility function to use the custom autograd function
def simulate(sensor_location, source_location, source_p0, source_dx, dt, num_sensors, num_sources, num_times):
    return SimulateFunction.apply(sensor_location, source_location, source_p0, source_dx, dt, num_sensors, num_sources, num_times)

In [3]:
import torch
import numpy as np
from torch.autograd import Function
from utils.loss_utils import l2_loss
import torch.optim as optim
import random
from utils.sliding_ball_model_fine import SlidingBallModel  
import matplotlib.pyplot as plt
import time
import os

device = torch.device('cuda:0')

def load_point_cloud(file_path):
    balls = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        start = False
        for line in lines:
            if start:
                data = line.split()
                if len(data) == 5:
                    xyz = torch.tensor([float(data[0]), float(data[1]), float(data[2])], requires_grad=True, dtype=torch.float32, device=device)
                    pressure_0 = torch.tensor(float(data[3]), requires_grad=True, dtype=torch.float32, device=device)
                    radius = torch.tensor(float(data[4]), requires_grad=True, dtype=torch.float32, device=device)
                    ball = SlidingBallModel(xyz, pressure_0, radius)
                    balls.append(ball)
            if 'end_header' in line:
                start = True
    return balls

file_path = 'point_cloud_coarse/ball_300_after.ply'
balls = load_point_cloud(file_path)
print(f"{len(balls)} balls loaded from point cloud file.")

# 随机初始化球分布
def generate_random_balls(num_balls, boundaries, res):
    balls = []
    x_min, x_max, y_min, y_max, z_min, z_max = boundaries
    for _ in range(num_balls):
        xyz = torch.tensor([random.uniform(x_min, x_max), random.uniform(y_min, y_max), random.uniform(z_min, z_max)], requires_grad=False, dtype=torch.float32, device=device)
        pressure_0 = torch.tensor(random.uniform(20, 100), requires_grad=True, dtype=torch.float32, device=device)
        radius = torch.tensor(random.uniform(1*res, 6*res), requires_grad=True, dtype=torch.float32, device=device)
        ball = SlidingBallModel(xyz, pressure_0, radius)
        balls.append(ball)
    return balls

# 迭代函数
def run_iterations(balls, pressure_threshold, radius_max_threshold, radius_min_threshold, boundaries):
    new_balls = []
    for ball in balls:
        new_ball = ball.adaptive_density_optimization(pressure_threshold, radius_max_threshold, radius_min_threshold, boundaries)
        if new_ball is not None:
            new_balls.append(new_ball)
    balls = [ball for ball in balls if not ball._is_destroyed] + new_balls
    return balls

# 保存点云数据函数
def save_point_cloud(balls, filename):
    points = []
    for ball in balls:
        if not ball._is_destroyed:
            xyz = ball._xyz.detach().cpu().numpy()  # 转为 numpy 数组
            pressure_0 = ball._pressure_0.item()  # 提取标量值
            radius = ball._radius.item()  # 提取标量值
            points.append([xyz[0], xyz[1], xyz[2], pressure_0, radius])

    with open(filename, "w") as ply_file:
        # 写 PLY 文件头
        ply_file.write("ply\n")
        ply_file.write("format ascii 1.0\n")
        ply_file.write(f"element vertex {len(points)}\n")
        ply_file.write("property float x\n")
        ply_file.write("property float y\n")
        ply_file.write("property float z\n")
        ply_file.write("property float pressure_0\n")
        ply_file.write("property float radius\n")
        ply_file.write("end_header\n")
        
        # 写入每个点的信息
        for point in points:
            ply_file.write(f"{point[0]} {point[1]} {point[2]} {point[3]} {point[4]}\n")

# 参数设置
num_times = 4096
Nt = num_times
res = 0.20e-3
Vs = 1500.0
dt = 25e-9  # [s]
folder_name = "point_cloud_fine"
boundaries = (0, 400 * res, 0, 520 * res, 0, 200 * res)
source_num = len(balls)
pressure_threshold = 15.0
radius_max_threshold = 3 * res
radius_min_threshold = 0.5 * res

c:\Users\ls\Desktop\SlingBAG\utils\sliding_ball_model_fine.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._xyz = Parameter(torch.tensor(xyz, dtype=torch.float32) if xyz is not None else torch.empty(0))
c:\Users\ls\Desktop\SlingBAG\utils\sliding_ball_model_fine.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._pressure_0 = Parameter(torch.tensor(pressure_0, dtype=torch.float32) if pressure_0 is not None else torch.empty(0))
c:\Users\ls\Desktop\SlingBAG\utils\sliding_ball_model_fine.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourc

80413 balls loaded from point cloud file.


In [4]:
# load sensor information
sensor_data_matrix = np.loadtxt('data/planar_196_sensor_data.txt', delimiter='\t')
real_signal = sensor_data_matrix
sensor_location = np.loadtxt('data/planar_196_sensor_location.txt', delimiter='\t')
sensor_location = sensor_location*1e-3
sensor_num = sensor_location.shape[0]
print(real_signal.shape)
print(sensor_location.shape)

(196, 4096)
(196, 3)


: 

In [ ]:
# 将数据转为PyTorch张量，并移动到GPU
sensor_location = torch.tensor(sensor_location,dtype=torch.float, device=device)
real_signal = torch.tensor(real_signal,dtype=torch.float)
real_signal_flat = real_signal.flatten()
real_signal_flat = real_signal_flat.to(device)

# 将所有参数按照不同学习率进行分组
def get_optim_params(balls):
    params_xyz = [ball._xyz for ball in balls]
    params_pressure = [ball._pressure_0 for ball in balls]
    params_radius = [ball._radius for ball in balls]
    return [
        {'params': params_xyz, 'lr': 0.000004},
        {'params': params_pressure, 'lr': 0.004},
        {'params': params_radius, 'lr': 0.000004},
    ]

optimizer = optim.Adam(get_optim_params(balls), betas=(0.9, 0.999))

# 设置学习率调度器
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

num_iterations = 1000

# 开始迭代训练
for iter in range(num_iterations):
    start_time = time.time()  # 记录开始时间
    optimizer.zero_grad()

    # 使用当前球的参数传递给simulate函数
    source_location = torch.stack([ball._xyz for ball in balls]).flatten()
    source_p0 = torch.stack([ball._pressure_0 for ball in balls]).flatten()
    radius_0 = torch.stack([ball._radius for ball in balls]).flatten()
    
    simulate_record = simulate(sensor_location, source_location, source_p0, radius_0, dt, sensor_num, source_num, num_times)  
    loss = l2_loss(simulate_record, real_signal_flat)
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    end_time = time.time()  # 记录结束时间
    iteration_time = end_time - start_time  # 计算时间差
    print(f"迭代 {iter + 1}，损失: {loss.item()}，时间: {iteration_time}")
    # 更新每个球的参数，以确保它们在相应的实例中
    for ball in balls:
        ball._xyz.data = ball._xyz.data.requires_grad_(True)
        ball._pressure_0.data = ball._pressure_0.data.requires_grad_(True)
        ball._radius.data = ball._radius.data.requires_grad_(True)

    # 每10次迭代前保存点云数据到文件
    if (iter + 1) % 10 == 0:
        filename = f"ball_{iter + 1}_before.ply"
        full_path = os.path.join(folder_name, filename)
        save_point_cloud(balls, full_path)
        print(f"Point cloud saved to {filename}")

    # 每20次迭代进行自适应密度优化
    if (iter + 1) % 20 == 0:
        with torch.no_grad():
            # 更新每个球的梯度状态，为下一次分裂做准备
            for ball in balls:
                ball._xyz.grad = ball._xyz.grad / torch.norm(ball._xyz.grad, p=2) if ball._xyz.grad is not None else None

            balls = run_iterations(balls, pressure_threshold, radius_max_threshold, radius_min_threshold, boundaries)
            source_num = len(balls)  # 更新 source_num 为当前球的数量

        # 打印当前球的数量
        print(f"After {iter + 1} iterations, number of balls: {source_num}")

        # 更新优化器中的参数
        optimizer = optim.Adam(get_optim_params(balls), betas=(0.9, 0.999))
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

    # 每10次迭代后保存点云数据到文件
    if (iter + 1) % 10 == 0:
        filename = f"ball_{iter + 1}_after.ply"
        full_path = os.path.join(folder_name, filename)
        save_point_cloud(balls, full_path)
        print(f"Point cloud saved to {filename}")

    if (iter + 1) % 100 == 40:
        with torch.no_grad():
            new_balls = []
            for ball in balls:
                if ball._xyz.grad is not None:
                    gradient_direction = ball._xyz.grad
                    new_ball = ball.clone_along_gradient(gradient_direction, boundaries,res)
                    if new_ball is not None:
                        new_balls.append(new_ball)
            balls.extend(new_balls)
            source_num = len(balls)  # 更新 source_num 为当前球的数量

        # 打印当前球的数量
        print(f"After {iter + 1} iterations with cloning, number of balls: {source_num}")

        # 更新优化器中的参数
        optimizer = optim.Adam(get_optim_params(balls), betas=(0.9, 0.999))
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
        # 保存点云
        filename = f"ball_{iter + 1}_after_duplication.ply"
        full_path = os.path.join(folder_name, filename)
        save_point_cloud(balls, full_path)
        print(f"Point cloud saved to {filename}")

迭代 1，损失: 288.9027404785156，时间: 269.38558626174927
